In [1]:
print("jay ganesh")

jay ganesh


In [2]:
import cv2

def extract_frames(video_path, frame_rate=1):
    frames = []
    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = max(1, fps // frame_rate)
    count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_interval == 0:
            frames.append(frame)
        count += 1
    cap.release()
    return frames


In [3]:
from moviepy.video.io.VideoFileClip import VideoFileClip

def extract_audio(video_path):
    clip = VideoFileClip(video_path)
    audio_path = "audio.wav"
    clip.audio.write_audiofile(audio_path)
    return audio_path


In [4]:
import speech_recognition as sr

def audio_to_text(audio_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)
        transcript = recognizer.recognize_google(audio_data)
    return transcript


In [5]:
import speech_recognition as sr

def audio_to_text(audio_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)
        try:
            # Add the correct language code if needed
            transcript = recognizer.recognize_google(audio_data, language="en-US")
        except sr.UnknownValueError:
            transcript = "[Unintelligible]"
        except sr.RequestError as e:
            transcript = f"[API error: {e}]"
    return transcript


In [6]:
import librosa

def sound_frequency_analysis(audio_path):
    y, sr = librosa.load(audio_path)
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
    # Example threshold
    return 1 if spectral_centroid > 3000 else 0


In [7]:
def angle_wise_analysis(frames):
    # Detect specific patterns (e.g., straight angles, zooms, pans)
    # Example placeholder logic
    return 0.8  # Acting-like probability


In [8]:
def cut_scene_detection(frames):
    cut_count = 0
    for i in range(1, len(frames)):
        diff = cv2.absdiff(cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY),
                           cv2.cvtColor(frames[i - 1], cv2.COLOR_BGR2GRAY)).mean()
        if diff > 50:  # Threshold for scene change
            cut_count += 1
    return cut_count / len(frames)


In [9]:
def motion_detection(frames):
    motion_score = 0
    for i in range(1, len(frames)):
        flow = cv2.calcOpticalFlowFarneback(
            cv2.cvtColor(frames[i - 1], cv2.COLOR_BGR2GRAY),
            cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY),
            None, 0.5, 3, 15, 3, 5, 1.2, 0
        )
        motion_score += flow.mean()
    return motion_score / len(frames)


In [10]:
def stability_analysis(frames):
    stability_score = 0
    for i in range(1, len(frames)):
        diff = cv2.absdiff(frames[i], frames[i - 1]).mean()
        stability_score += diff
    return 1 - (stability_score / len(frames))  # Invert for stability score


In [11]:
from transformers import pipeline

def script_based_classification(transcript):
    classifier = pipeline("text-classification", model="distilbert-base-uncased")
    result = classifier(transcript)
    return 1 if result[0]['label'] == 'POSITIVE' else 0


c:\Users\ipatel\AppData\Local\anaconda3\envs\vclass\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def classify_scene(video_path):
    frames = extract_frames(video_path)
    audio_path = extract_audio(video_path)
    transcript = audio_to_text(audio_path)

    sound_prob = sound_frequency_analysis(audio_path)
    angle_prob = angle_wise_analysis(frames)
    cut_scene_prob = cut_scene_detection(frames)
    motion_prob = motion_detection(frames)
    stability_prob = stability_analysis(frames)
    script_prob = script_based_classification(transcript)

    # Combine probabilities
    weights = [0.2, 0.2, 0.2, 0.2, 0.1, 0.1]  # Adjust as needed
    probs = [sound_prob, angle_prob, cut_scene_prob, motion_prob, stability_prob, script_prob]
    final_prob = sum(w * p for w, p in zip(weights, probs))

    return "Acting" if final_prob > 0.5 else "Real-Life"


In [19]:
video_path = r"D:\video_classification\video_classification\fake.mp4"
result = classify_scene(video_path)
print(f"The scene is classified as: {result}")

{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'mp42', 'minor_version': '0', 'compatible_brands': 'isommp42', 'encoder': 'Google'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [512, 640], 'bitrate': 567, 'fps': 30.0, 'codec_name': 'h264', 'profile': '(Constrained Baseline)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 48000, 'bitrate': 192, 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 59.99, 'bitrate': 761, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'h264', 'video_profile': '(Constrained Baseline)', 'video_size': [512, 640], 'video_bitra

MoviePy - Done.


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


The scene is classified as: Real-Life


In [15]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())  # Optional: Check if GPU is available


2.5.1+cpu
False


In [17]:
from transformers import pipeline
classifier = pipeline("text-classification", model="distilbert-base-uncased")
print("Model loaded successfully!")


c:\Users\ipatel\AppData\Local\anaconda3\envs\vclass\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ipatel\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForSequenceClassification were not initialized from the model check

Model loaded successfully!
